# 序列模型
:label:`sec_sequence`

假设您正在Netflix上观看电影。作为一个优秀的Netflix用户，你决定对每一部电影进行严格的评级。
毕竟，一部好电影就是一部好电影，你想看更多的，对吗？事实证明，事情并不是那么简单。
随着时间的推移，人们对电影的看法会发生很大的变化。事实上，心理学家甚至对某些效应有了名字：

* 有一种“锚定”，基于其他人的观点。例如，奥斯卡颁奖后，相应电影的收视率上升，尽管它仍然是同一部电影。这种影响持续几个月，直到该奖项被遗忘。结果表明，该效应将评级提高了半个百分点以上
:cite:`Wu.Ahmed.Beutel.ea.2017`.
* 有一种*享乐适应*，即人类迅速适应，接受改善或恶化的情况作为新常态。例如，在看了很多好电影之后，人们对下一部电影同样好或更好的期望很高。因此，即使是一部普通的电影，在看了很多伟大的电影之后，也可能被认为是糟糕的。
* 有*季节性*。很少有观众喜欢在八月看圣诞老人的电影。
* 在某些情况下，由于导演或演员在制作中的不当行为，电影变得不受欢迎。
* 有些电影变成了邪教电影，因为它们差得近乎滑稽*太空计划9*和*食人魔*因此声名狼藉。

简言之，电影收视率绝不是固定不变的。因此，使用时间动力学
导致了更准确的电影推荐：:cite:`Koren.2009`。
当然，序列数据不仅仅与电影收视率有关。下面给出了更多的说明。

* 许多用户在打开应用程序时都有非常特殊的行为。例如，社交媒体应用在学生放学后更受欢迎。股市交易应用程序在市场开放时更常用。
* 预测明天的股票价格要比填补我们昨天错过的股票价格的空白困难得多，尽管两者都只是估计一个数字。毕竟，远见卓识远比事后诸葛亮难。在统计学中，前者（在已知观测值之外进行预测）称为*外推*，而后者（在现有观测值之间进行估计）称为*内插*。
* 音乐、语音、文本和视频本质上都是连续的。如果我们把他们换掉，他们就毫无意义了。标题*狗咬人*远没有*人咬狗*那么令人惊讶，尽管两个词完全相同。
* 地震具有很强的相关性，即，在大地震之后，很可能会有几次较小的余震，比没有强震的情况下更为明显。事实上，地震是时空相关的，也就是说，余震通常发生在很短的时间跨度内并且非常接近。
* 人类之间的互动是连续的，这可以从推特上的争斗、舞蹈模式和辩论中看出。


## 统计工具

我们需要统计工具和新的深层神经网络结构来处理序列数据。为了简单起见，我们以股票价格 (FTSE 100 index) 为例，如
:numref:`fig_ftse100` 所示。

![FTSE 100 index over about 30 years.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/ftse100.png)
:width:`400px`
:label:`fig_ftse100`

让我们用 $x_t$ 表示价格，即在 *时间步长* $t \in \mathbb{Z}^+$ 我们观察到的价格 $x_t$ 。
请注意，对于本文中的序列，
$t$ 通常是离散的，在整数或其子集上变化。
假设
想在 $t$ 交易日在股市表现出色的交易员通过via预测 $x_t$

$$x_t \sim P(x_t \mid x_{t-1}, \ldots, x_1).$$

### 自回归模型

为了实现这一点，我们的交易者可以使用一个回归模型，比如我们训练的
:numref:`sec_linear_concise`.
只有一个主要问题：输入的数量， $x_{t-1}, \ldots, x_1$ 因 $t$ 而异。
也就是说，这个数字随着我们遇到的数据量的增加而增加，我们需要一个近似值，使这个计算变得容易处理。
本章后面的大部分内容将围绕如何有效地估算 $P(x_t \mid x_{t-1}, \ldots, x_1)$ 。
简而言之，它归结为以下两种策略。

首先，假设可能相当长的序列 $x_{t-1}, \ldots, x_1$ 实际上不是必需的。
在这种情况下，我们可能满足于长度为 $\tau$ 的时间跨度，并且只使用 $x_{t-1}, \ldots, x_{t-\tau}$ 观察值。
直接的好处是现在参数的数量总是相同的，至少对于 $t > \tau$。这使我们能够训练如上所述的深度网络。
这种模型将被称为*自回归模型*，因为它们实际上是在自己身上进行回归的。

第二种策略，如 :numref:`fig_sequence-model`, 所示，是保留过去观测的一些摘要 $h_t$ 同时除了预测 $\hat{x}_t$ 之外更新 $h_t$ 
这导致了用 $x_t$ with $\hat{x}_t = P(x_t \mid h_{t})$ 的模型，并且更新了 $h_t = g(h_{t-1}, x_{t-1})$ 的形式。
由于从未观测到 $h_t$ ，这些模型也称为*潜在自回归模型*。

![一个潜在的自回归模型。](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/sequence-model.svg)
:label:`fig_sequence-model`

这两种情况都提出了如何生成训练数据的明显问题。人们通常使用历史观测来预测下一次观测，给出目前为止的观测结果。
显然，我们不指望时间静止不动。然而，一个常见的假设是，虽然$x_t$ 的特定值可能会改变，但至少序列本身的动态不会改变。
这是合理的，因为新的动力学就是这样，新的，因此不可预测使用数据，我们到目前为止。
统计学家把不变的动态称为*静止*。
不管我们做什么，我们都将通过

$$P(x_1, \ldots, x_T) = \prod_{t=1}^T P(x_t \mid x_{t-1}, \ldots, x_1).$$

请注意，如果我们处理离散对象，例如单词，而不是连续数字，上述考虑仍然有效。唯一的区别是，
在这种情况下，我们需要使用分类器而不是回归模型来估计 
$P(x_t \mid  x_{t-1}, \ldots, x_1)$ 。

### 马尔可夫模型

回想一下，在自回归模型中，我们只使用 $x{t-1}、\ldots，x{t-\tau}$ 而不是 $x{t-1}, \ldots, x_{t-\tau}$ 而不是 $x_{t-1}, \ldots, x_1$ 来估计 $x_t$ 。只要这个近似是精确的，我们就说序列满足一个马尔可夫条件。特别是，如果 $\tau = 1$ ，我们有一个*一阶马尔可夫模型*，并且 $P(x)$ 由

$$P(x_1, \ldots, x_T) = \prod_{t=1}^T P(x_t \mid x_{t-1}) \text{ where } P(x_1 \mid x_0) = P(x_1).$$

只要 $x_t$ 只假设一个离散值，这种模型就特别好，因为在这种情况下，动态规划可以用来精确计算链上的值。
例如，我们可以有效地计算 $P(x_{t+1} \mid x_{t-1})$ ：


$$\begin{aligned}
P(x_{t+1} \mid x_{t-1})
&= \frac{\sum_{x_t} P(x_{t+1}, x_t, x_{t-1})}{P(x_{t-1})}\\
&= \frac{\sum_{x_t} P(x_{t+1} \mid x_t, x_{t-1}) P(x_t, x_{t-1})}{P(x_{t-1})}\\
&= \sum_{x_t} P(x_{t+1} \mid x_t) P(x_t \mid x_{t-1})
\end{aligned}
$$

通过使用我们只需要考虑过去观测的非常短的历史这一事实：$P(x_{t+1} \mid x_t, x_{t-1}) = P(x_{t+1} \mid x_t)$。
深入了解动态规划的细节超出了本节的范围。控制和强化学习算法广泛使用此类工具。

### 因果关系

原则上，以相反的顺序展开 $P(x_1, \ldots, x_T)$ 没有错。毕竟，通过条件作用，我们总是可以通过

$$P(x_1, \ldots, x_T) = \prod_{t=T}^1 P(x_t \mid x_{t+1}, \ldots, x_T).$$

事实上，如果我们有一个马尔可夫模型，我们也可以得到一个反向的条件概率分布。然而，在许多情况下，数据存在一个自然的方向，即及时向前。
很明显，未来的事件不能影响过去。因此，如果我们更改 $x_t$ ，我们可能会影响 $x_{t+1}$ 的未来情况，而不是相反。
也就是说，如果我们改变 $x_t$ ，过去事件的分布不会改变。
因此，解释 $P(x_{t+1} \mid x_t)$ 应该比解释 $P(x_t \mid x_{t+1})$ 更容易。
例如，已经证明，在某些情况下，对于某些加性噪声 $\epsilon$ 我们可以找到 $x_{t+1} = f(x_t) + \epsilon$ ，
反之则不成立 :cite:`Hoyer.Janzing.Mooij.ea.2009`。这是个好消息，因为这通常是我们感兴趣估计的前进方向。
Peters等人 的书已经出版了
关于这个话题的更多解释 :cite:`Peters.Janzing.Scholkopf.2017`。
我们几乎没有触及它的表面。


## 训练

在回顾了这么多统计工具之后，
让我们在实践中尝试一下。
我们首先生成一些数据。
为了简单起见，我们使用正弦函数生成序列数据，该函数带有一些时间步长 $1, 2, \ldots, 1000$。


In [ ]:
%load ../utils/djl-imports
%load ../utils/plot-utils
%load ../utils/Functions.java

In [ ]:
NDManager manager = NDManager.newBaseManager();

In [ ]:
public static Figure plot(double[] x, double[] y, String xLabel, String yLabel) {
    ScatterTrace trace = ScatterTrace.builder(x,y)
        .mode(ScatterTrace.Mode.LINE)
        .build();

    Layout layout = Layout.builder()
            .showLegend(true)
            .xAxis(Axis.builder().title(xLabel).build())
            .yAxis(Axis.builder().title(yLabel).build())
            .build();

    return new Figure(layout, trace);
}

In [ ]:
int T = 1000; // 总共生成1000个点
NDArray time = manager.arange(1f, T+1);
NDArray x = time.mul(0.01).sin().add(
    manager.randomNormal(0f, 0.2f, new Shape(T), DataType.FLOAT32));

double[] xAxis = Functions.floatToDoubleArray(time.toFloatArray());
double[] yAxis = Functions.floatToDoubleArray(x.toFloatArray());

plot(xAxis, yAxis, "time", "x");

接下来，我们需要将这样一个序列转换为我们的模型可以训练的特征和标签。
基于嵌入维度 $\tau$ ，我们将数据映射到 $y_t = x_t$ 和$\mathbf{x}_t = [x_{t-\tau}, \ldots, x_{t-1}]$ 对中。
精明的读者可能已经注意到，这给我们提供了 $\tau$ 更少的数据示例，因为我们没有足够的第一个 $\tau$ 的历史记录。
一个简单的修复，特别是如果序列很长，
就是放弃这几个术语。
或者我们可以用零填充序列。
这里，我们仅使用前600个特征标签对进行训练。


In [ ]:
int tau = 4;
NDArray features = manager.zeros(new Shape(T - tau, tau));

for (int i = 0; i < tau; i++) {
    features.set(new NDIndex(":, {}", i), x.get(new NDIndex("{}:{}", i, T - tau + i)));
}
NDArray labels = x.get(new NDIndex("" + tau + ":")).reshape(new Shape(-1,1));

In [ ]:
int batchSize = 16;
int nTrain = 600;
// 只有第一个“nTrain”示例用于训练
ArrayDataset trainIter = new ArrayDataset.Builder()
    .setData(features.get(new NDIndex(":{}", nTrain)))
    .optLabels(labels.get(new NDIndex(":{}", nTrain)))
    .setSampling(batchSize, true)
    .build();

在这里，我们使体系结构相当简单：
只有两个完全连接层的MLP，ReLU激活和平方损耗。


In [ ]:
// 一个简单的MLP
public static SequentialBlock getNet() {
    SequentialBlock net = new SequentialBlock();
    net.add(Linear.builder().setUnits(10).build());
    net.add(Activation::relu);
    net.add(Linear.builder().setUnits(1).build());
    return net;
}

现在我们已经准备好训练模型了。下面的代码与前面章节中的培训循环基本相同，
例如 :numref:`sec_linear_concise`.
因此，我们将不深入探讨太多细节。


In [ ]:
// 我们在函数 "train" 之外添加此项，以便在笔记本中保留对训练对象的强引用（否则有时可能会关闭）
Trainer trainer = null; 

In [ ]:
public static Model train(SequentialBlock net, ArrayDataset dataset, int batchSize, int numEpochs, float learningRate) 
    throws IOException, TranslateException {
    // 平方误差
    Loss loss = Loss.l2Loss();
    Tracker lrt = Tracker.fixed(learningRate);
    Optimizer adam = Optimizer.adam().optLearningRateTracker(lrt).build();
    
    DefaultTrainingConfig config = new DefaultTrainingConfig(loss)
        .optOptimizer(adam) // 优化器（损失函数）
        .optInitializer(new XavierInitializer(), "")
        .addTrainingListeners(TrainingListener.Defaults.logging()); // 日志
    
    Model model = Model.newInstance("sequence");
    model.setBlock(net);
    trainer = model.newTrainer(config);
    
    for (int epoch = 1; epoch <= numEpochs; epoch++) {
        // 在数据集上迭代
        for (Batch batch : trainer.iterateDataset(dataset)) {
            // 更新损失率和评估器
            EasyTrain.trainBatch(trainer, batch);

            // 更新参数
            trainer.step();

            batch.close();
        }
        
        // 在新epoch结束时重置训练和验证求值器
        trainer.notifyListeners(listener -> listener.onEpoch(trainer));
        System.out.printf("Epoch %d\n", epoch);
        System.out.printf("Loss %f\n", trainer.getTrainingResult().getTrainLoss());
        
        
    }
    return model;
}
SequentialBlock net = getNet();
Model model = train(net, trainIter, batchSize, 5, 0.01f);

## 预测

由于训练损失很小，我们希望我们的模型能够很好地工作。
让我们看看这在实践中意味着什么。
首先要检查的是模型能够预测下一个时间步发生的情况，
即*提前一步预测*。


In [ ]:
Translator translator = new NoopTranslator(null);
Predictor predictor = model.newPredictor(translator);

NDArray onestepPreds = ((NDList) predictor.predict(new NDList(features))).get(0);

ScatterTrace trace = ScatterTrace.builder(Functions.floatToDoubleArray(time.toFloatArray()), 
                                          Functions.floatToDoubleArray(x.toFloatArray()))
        .mode(ScatterTrace.Mode.LINE)
        .name("data")
        .build();

ScatterTrace trace2 = ScatterTrace.builder(Functions.floatToDoubleArray(time.get(new NDIndex("{}:", tau)).toFloatArray()),
                                           Functions.floatToDoubleArray(onestepPreds.toFloatArray()))
        .mode(ScatterTrace.Mode.LINE)
        .name("1-step preds")
        .build();

Layout layout = Layout.builder()
        .showLegend(true)
        .xAxis(Axis.builder().title("time").build())
        .yAxis(Axis.builder().title("x").build())
        .build();

new Figure(layout, trace, trace2);

正如我们预期的那样，提前一步的预测看起来不错。
即使超过604次（`n_train+tau`）观测，这些预测看起来仍然可信。
然而，这里只有一个小问题：
如果我们只在时间步604之前观察序列数据，我们就不可能希望收到所有未来一步预测的输入。
相反，我们需要一步一步地向前迈进：

$$
\hat{x}_{605} = f(x_{601}, x_{602}, x_{603}, x_{604}), \\
\hat{x}_{606} = f(x_{602}, x_{603}, x_{604}, \hat{x}_{605}), \\
\hat{x}_{607} = f(x_{603}, x_{604}, \hat{x}_{605}, \hat{x}_{606}),\\
\hat{x}_{608} = f(x_{604}, \hat{x}_{605}, \hat{x}_{606}, \hat{x}_{607}),\\
\hat{x}_{609} = f(\hat{x}_{605}, \hat{x}_{606}, \hat{x}_{607}, \hat{x}_{608}),\\
\ldots
$$

通常，对于高达 $x_t$ 的观测序列，其在时间步长 $t+k$ 处的预测输出 $\hat{x}_{t+k}$ 
称为*$k$-步进预测*。
由于我们观察到高达 $x_{604}$，其 $k$-步骤-预测为 $\hat{x}_{604+k}$。
换句话说，我们将不得不使用我们自己的预测来进行多步预测。
让我们看看进展如何。


In [ ]:
NDArray multiStepPreds = manager.zeros(new Shape(T));
multiStepPreds.set(new NDIndex(":{}", nTrain + tau), x.get(new NDIndex(":{}", nTrain + tau)));
for (int i = nTrain + tau; i < T; i++) {
    NDArray tempX = multiStepPreds.get(new NDIndex("{}:{}", i - tau, i)).reshape(new Shape(1, -1));
    NDArray prediction = ((NDList) predictor.predict(new NDList(tempX))).get(0);
    multiStepPreds.set(new NDIndex(i), prediction);
}

ScatterTrace trace = ScatterTrace.builder(Functions.floatToDoubleArray(time.toFloatArray()), 
                                          Functions.floatToDoubleArray(x.toFloatArray()))
        .mode(ScatterTrace.Mode.LINE)
        .name("data")
        .build();

ScatterTrace trace2 = ScatterTrace.builder(Functions.floatToDoubleArray(time.get(new NDIndex("{}:", tau)).toFloatArray()),
                                           Functions.floatToDoubleArray(onestepPreds.toFloatArray()))
        .mode(ScatterTrace.Mode.LINE)
        .name("1-step preds")
        .build();

ScatterTrace trace3 = ScatterTrace.builder(Functions.floatToDoubleArray(time.get(
                                                new NDIndex("{}:", nTrain + tau)).toFloatArray()),
                                           Functions.floatToDoubleArray(multiStepPreds.get(
                                               new NDIndex("{}:", nTrain + tau)).toFloatArray()))
        .mode(ScatterTrace.Mode.LINE)
        .name("multistep preds")
        .build();

Layout layout = Layout.builder()
        .showLegend(true)
        .xAxis(Axis.builder().title("time").build())
        .yAxis(Axis.builder().title("x").build())
        .build();

new Figure(layout, trace, trace2, trace3);

正如上面的例子所示，这是一个惊人的失败。经过几个预测步骤后，预测很快衰减为常数。
为什么算法工作得这么差？
这最终是由于错误累积的事实。
假设在第1步之后，出现了一些错误 $\epsilon_1 = \bar\epsilon$。
现在，步骤2的*输入*被 $\epsilon_1$ 扰动，因此对于某些常量 $c$ ，
我们会遇到一些顺序为 $\epsilon_2 = \bar\epsilon + c \epsilon_1$ 的错误，依此类推。
误差可能很快偏离真实观测值。
这是一种普遍现象。
例如，未来24小时的天气预报往往相当准确，但除此之外，准确度会迅速下降。
我们将在本章及以后讨论改进方法。

让我们仔细看看 $k$-超前 预测中的困难
通过计算对 $k = 1, 4, 16, 64$ 的整个序列的预测。


In [ ]:
int maxSteps = 64;

NDArray features = manager.zeros(new Shape(T - tau - maxSteps + 1, tau + maxSteps));
// 列 `i` (`i` < `tau`) 是从 `x` 观察到的来自
// `i + 1` 到 `i + T - tau - maxSteps + 1`
for (int i = 0; i < tau; i++) {
    features.set(new NDIndex(":, {}", i), x.get(new NDIndex("{}:{}", i, i + T - tau - maxSteps + 1)));
}
// 列 `i` (`i` >= `tau`) 是 (`i - tau + 1`) 的超前预测
// 从 `i + 1` to `i + T - tau - maxSteps + 1` 的时间步长
for (int i = tau; i < tau + maxSteps; i++) {
    NDArray tempX = features.get(new NDIndex(":, {}:{}", i - tau, i));
    NDArray prediction = ((NDList) predictor.predict(new NDList(tempX))).get(0);
    features.set(new NDIndex(":, {}", i), prediction.reshape(-1));
}

In [ ]:
int[] steps = new int[] {1, 4, 16, 64};

ScatterTrace[] traces = new ScatterTrace[4];

for (int i = 0; i < traces.length; i++) {
    int step = steps[i];
    traces[i] = ScatterTrace.builder(Functions.floatToDoubleArray(time.get(new NDIndex("{}:{}", tau + step - 1, T - maxSteps + i)).toFloatArray()), 
                                     Functions.floatToDoubleArray(features.get(
                                         new NDIndex(":,{}", tau + step - 1)).toFloatArray())
                                    )
                .mode(ScatterTrace.Mode.LINE)
                .name(step + "-step preds")
                .build();
}


Layout layout = Layout.builder()
                .showLegend(true)
                .xAxis(Axis.builder().title("time").build())
                .yAxis(Axis.builder().title("x").build())
                .build();

new Figure(layout, traces);

这清楚地说明了当我们试图预测未来时，预测的质量是如何变化的。
尽管提前4步的预测看起来仍然不错，但超出这一步的任何预测几乎都是无用的。


## 总结

* 内插法和外推法在难度上有很大差别。因此，如果您有一个序列，那么在进行训练时，请始终遵守数据的时间顺序，即永远不要对未来的数据进行训练。
* 序列模型需要专门的统计工具进行估计。两种流行的选择是自回归模型和潜变量自回归模型。
* 对于因果模型（例如，时间向前），估计正向通常比反向容易得多。
* 对于时间步 $t$ 之前的观测序列，其在时间步 $t+k$ 的预测输出为 $k$*-超前预测*。当我们通过增加 $k$ 来进一步预测时，误差会累积，预测质量会下降，通常会急剧下降。


## 练习

1. 在本部分的实验中对模型进行了改进。
    1. 纳入超过过去4次的观察结果？你到底需要多少？
    2. 如果没有noise，您需要多少过去的观察？提示：您可以将 $\sin$ 和 $\cos$ 写成微分方程。
    3. 在保持特征总数不变的情况下，您能否合并旧的观察结果？这会提高准确性吗？为什么？
    4. 改变神经网络结构并评估性能。
2. 投资者希望找到一种好的证券来购买。他查看过去的回报，以决定哪一个可能表现良好。这种策略可能会出什么问题？
3. 因果关系是否也适用于文本？到什么程度？
4. 举例说明何时可能需要潜在自回归模型来捕获数据的动态。
